# KOBERT 모델로 감성분류(7가지 감성)

## 데이터 로드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/BOAZ_miniProject2/sentiment_data.csv')

In [3]:
data.head()

,Unnamed: 0,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female
1,1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness,neutral,0,happiness,2,happiness,2,happiness,2,happiness,2,48,female
2,2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female
3,3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,1,48,female
4,4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral,happiness,2,happiness,1,happiness,2,happiness,1,happiness,1,48,female


## EDA

In [ ]:
data.shape

(43991, 16)

In [ ]:
data.wav_id.nunique() # 겹치는 값 없음

43991

In [ ]:
print("최소 길이: ", min(data.발화문.map(lambda x: len(x)).tolist()))
print("최대 길이: ", max(data.발화문.map(lambda x: len(x)).tolist()))

최소 길이:  1
최대 길이:  137


In [4]:
# 상황열 감정을 숫자로 인코딩
data.loc[(data['상황'] == "fear"), '상황'] = 0  #공포 => 0
data.loc[(data['상황'] == "surprise"), '상황'] = 1  #놀람 => 1
data.loc[(data['상황'] == "angry"), '상황'] = 2  #분노 => 2
data.loc[(data['상황'] == "anger"), '상황'] = 2  #분노 => 2
data.loc[(data['상황'] == "sadness"), '상황'] = 3  #슬픔 => 3
data.loc[(data['상황'] == "sad"), '상황'] = 3  #슬픔 => 3
data.loc[(data['상황'] == "neutral"), '상황'] = 4  #중립 => 4
data.loc[(data['상황'] == "happiness"), '상황'] = 5  #행복 => 5
data.loc[(data['상황'] == "disgust"), '상황'] = 6  #혐오 => 6

In [ ]:
# 감정 분포
data.상황.value_counts()

3    14000
2    11635
6     4660
5     4548
0     4131
4     3262
1     1755
Name: 상황, dtype: int64

In [ ]:
data.head()

,Unnamed: 0,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,5,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female
1,1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,5,neutral,0,happiness,2,happiness,2,happiness,2,happiness,2,48,female
2,2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",5,angry,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female
3,3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",5,angry,2,happiness,2,happiness,2,happiness,2,happiness,1,48,female
4,4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,4,happiness,2,happiness,1,happiness,2,happiness,1,happiness,1,48,female


## Modelling

### 환경설정

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-n22v4wr0/kobert-tokenizer_1adf216598484163b4ead98fa483a107
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-n22v4wr0/kobert-tokenizer_1adf216598484163b4ead98fa483a107
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=65db0052fa1cd0eea4719443d42e171c315213046f1b7e8c7bd55c9afb416a7c
  Stored in directory: /tmp/pip-ephem-wheel-cache-321cke2p/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [6]:
!pip install transformers==4.25.1
!pip install --no-cache-dir transformers sentencepiece
!pip install mxnet
!pip install gluonnlp==0.9.1
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
import transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

### Hugingface에서 model, tokenizer 불러오기

In [8]:
def get_kobert_model(model_path, vocab_file, ctx="cuda"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [10]:
# 사전학습 모델 불러오기
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


### Dataset 설정

In [11]:
data_list = []
for ques, label in zip(data['발화문'], data['상황']):
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [12]:
data_list

[['헐! 나 이벤트에 당첨 됐어.', '5'],
 ['내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.', '5'],
 ['한 명 뽑는 거였는데, 그게 바로 내가 된 거야.', '5'],
 ['당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. 그래서 못 산 향수야.', '5'],
 ['에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.', '4'],
 ['난 부페 형식의 음식들도 정말 좋아해. 그 것도 좀 알려 줘.', '4'],
 ['응. 완전히 끝난 거야. 한 달 동안 주말에 쉬지도 못하고 일만 했거든.', '5'],
 ['신나는 음악 듣는 것도 좋고, 어디 여행 가고 싶고 이 것 저 것 다 해보고 싶어.', '5'],
 ['친구들도 내 연락 기다리고 있을 텐데 내가 까먹고 있었네?', '5'],
 ['그래. 일단은 친구들부터 만나서 여행 계획에 대해서 얘기 좀 해봐야 되겠어.', '5'],
 ['나 요즘 너무 우울해 죽겠어.', '3'],
 ['맞아. 사람은 나가서 활동을 해야 된다고.', '3'],
 ['좀 활기 있는 영화로 좀 부탁할게.', '3'],
 ['내가 좋아하는 유튜버가 좋은 제품이라고 하길래 나도 샀지. 그런데 그게 돈 받고 하는 광고였대잖아.', '2'],
 ['그래서 사람들이 증거 사진을 올렸더니 그 때서야 광고였다고 미안하다고 하는 거야.', '2'],
 ['처음부터 자기 잘못을 알고 미안하다고 했으면 화가 이렇게 나진 않았을 거야. 사람들이 증거를 올리니까 그 때서야 사과하는 게 진정성이 느껴지지가 않아.',
  '2'],
 ['아니. 이제부터 자랑 한번 해봐야지.', '5'],
 ['내 휴가가 엉망진창이 돼버렸어.', '2'],
 ['휴가 시작함과 동시에 장마도 같이 시작해버렸어. 그래서 하루 종일 비가 와.', '2'],
 ['여름 휴가는 뭐니뭐니 해도 여름 바다야. 그런데 지금 이렇게 비가 많이 오니 바다를 갈 수가 없어.', '2'],
 ['그 동안 바빠서 

In [13]:
# KoBert의 정수 인코딩
result = tokenizer.tokenize("너는 내년 대선 때 투표할 수 있어?")
print(result)
kobert_vocab = tokenizer.get_vocab()
print(kobert_vocab.get('▁대선'))
print([tokenizer.encode(token) for token in result])

['▁너', '는', '▁내년', '▁대선', '▁때', '▁투표', '할', '▁수', '▁있어', '?']
1654
[[2, 1457, 3], [2, 517, 5760, 3], [2, 1437, 3], [2, 1654, 3], [2, 1844, 3], [2, 4772, 3], [2, 4977, 3], [2, 2872, 3], [2, 3868, 3], [2, 633, 3]]


In [14]:
# 파라미터 설정
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [15]:
# train, test 데이터셋 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 42)

In [16]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length = max_len, vocab = vocab, pad = pad, pair = pair)

    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return (len(self.labels))

In [17]:
# train, test 리스트 형태를 BERT의 입력에 맞게 변환
train_data = BERTDataset(train, 0, 1, tokenizer.tokenize, vocab, max_len, True, False)
test_data = BERTDataset(test, 0, 1, tokenizer.tokenize, vocab, max_len, True, False)

In [18]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [19]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768, num_classes=7, #클래스 수 조정
                 dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
# BERT 모델 불러오기
device = torch.device("cuda:0")
model = BERTClassifier(model,  dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Training

In [21]:
train_history=[]
test_history=[]
loss_history=[]

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-21-fbbe4ae2ca37>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/550 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.04703950881958 train acc 0.15625
epoch 1 batch id 201 loss 0.7961373925209045 train acc 0.46097636815920395
epoch 1 batch id 401 loss 0.31757858395576477 train acc 0.6489245635910225
epoch 1 train acc 0.7120982142857143


<ipython-input-21-fbbe4ae2ca37>:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/138 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9239057386629266


  0%|          | 0/550 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.3448081910610199 train acc 0.875
epoch 5 batch id 201 loss 0.10853973031044006 train acc 0.9718594527363185
epoch 5 batch id 401 loss 0.01284086238592863 train acc 0.9739323566084788
epoch 5 train acc 0.9750487012987012


  0%|          | 0/138 [00:00<?, ?it/s]

epoch 5 test acc 0.9260570067788686


### Predict

In [22]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer.tokenize, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            print(logits)
            logits = logits.detach().cpu().numpy()
            print(logits)

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [24]:
# 질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 안녕
tensor([-0.5958, -0.6756, -1.3214, -0.4122,  6.2557, -1.0523, -0.7902],
       device='cuda:0', grad_fn=<UnbindBackward0>)
[-0.5957713  -0.6756136  -1.3213509  -0.4122174   6.2557416  -1.0522835
 -0.79016864]
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 나 오늘 기분이 좋아
tensor([-3.0175, -1.5182, -0.2255,  0.2639, -0.7363,  4.4367,  0.3978],
       device='cuda:0', grad_fn=<UnbindBackward0>)
[-3.0175254  -1.5182331  -0.22547899  0.2639357  -0.7363053   4.4366584
  0.39783287]
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 오늘 날씨가 어때?
tensor([-1.5314, -1.2569, -0.8154,  0.6693,  6.1516, -1.4637, -0.5732],
       device='cuda:0', grad_fn=<UnbindBackward0>)
[-1.5313996  -1.2569442  -0.8153962   0.6693005   6.151586   -1.4636774
 -0.57320523]
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 비가 오네
tensor([ 0.4638, -0.6420,  4.9088, -2.2774,  1.3375, -2.0201, -1.8708],
       device='cuda:0', grad_fn=<UnbindBackward0>)
[ 0.4637912  -0.64203465  4.908797   -2.2774096   1.3375229  -2.0201108


In [23]:
torch.save(model.state_dict(), '/content/drive/MyDrive/BOAZ_miniProject2/kobert.pth')